In [1]:
import numpy as np
import pandas as pd
from characters.sparkle import Sparkle
from characters.qingque import Qingque
from characters.silver_wolf import Silver_Wolf
from characters.fu_xuan import Fu_Xuan
from simulations.qingque_mono_quantum import QQ_Mono_Quantum, Initialize_QQ_Mono_Quantum
from utils.arena import Arena
from utils.data_table import Action_Table, Qingque_Probability_Table

In [2]:
qingque_speed = 135

In [3]:
silver_wolf_speed = 136

In [4]:
sparkle_speed = 134

In [5]:
fu_xuan_speed = 134
fu_xuan_technique = True

In [7]:
Initialize_QQ_Mono_Quantum(qingque_speed=qingque_speed, fu_xuan_speed=fu_xuan_speed, sparkle_speed=sparkle_speed, silver_wolf_speed=silver_wolf_speed)

(<utils.arena.Arena at 0x1a47a3b3c50>,
 <characters.fu_xuan.Fu_Xuan at 0x1a47a3b3c90>)

In [8]:
action_dataframe = Action_Table()
probability_dataframe = Qingque_Probability_Table()

In [10]:
QQ_Mono_Quantum(action_df=action_dataframe, prob_df= probability_dataframe)

c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)
c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)


In [11]:
pd.set_option("display.max_rows", None)
action_dataframe.df

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
0,Fu Xuan,0,100,6,67.50,Basic,1
1,Silver Wolf,0,100,7,55.00,Skill,-1
2,Sparkle,0,100,6,55.00,Skill,-1
3,Qingque,0.0,100.1,6,70.00,Enhanced Basic,1
4,Fu Xuan,1.0,200.0,5,91.38,Basic,1
5,Silver Wolf,1.0,200.0,6,55.00,Basic,1
6,Sparkle,1.0,200.0,7,90.82,Skill,-1
7,Sparkle,1.0,200.0,6,126.64,Ultimate,4
8,Qingque,1.0,200.1,7,70.00,Enhanced Basic,1
9,Fu Xuan,2.0,300.0,6,115.26,Skill,-1


In [12]:
probability_dataframe_styled = probability_dataframe.df.style.set_caption(f"Qingque speed = {qingque_speed}, Sparkle speed = {sparkle_speed}, Silver Wolf Speed = {silver_wolf_speed}, Fu Xuan speed = {fu_xuan_speed}")
probability_dataframe_styled

,Cycle,Action Gauge,Current SP,Current Tiles,P(Success),Average # Skills,Most Likely # Skills,SP Consumed
0,0.000000,100.100000,6,4,0.990547,1.861762,2.226879,2.000000
1,1.000000,200.100000,7,4,0.995798,1.898525,2.239724,2.000000
2,2.000000,300.100000,5,4,0.978734,1.790886,2.197988,2.000000
3,3.000000,400.100000,3,4,0.892812,1.420658,1.987654,2.000000
4,4.000000,500.100000,7,4,0.995798,1.898525,2.239724,2.000000
5,5.000000,600.100000,6,4,0.990547,1.861762,2.226879,2.000000
6,6.000000,700.100000,2,4,0.762222,1.028889,1.666667,2.000000
7,7.000000,800.100000,6,4,0.990547,1.861762,2.226879,2.000000
8,8.000000,900.100000,6,4,0.990547,1.861762,2.226879,2.000000
9,9.000000,1000.100000,4,4,0.952193,1.658183,2.133059,2.000000
